<a href="https://colab.research.google.com/github/sakibreza/Emotion-Recognition-from-Multi-Channel-EEG-Signals/blob/main/pytorch_nn_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
import pickle as pickle
import matplotlib.pyplot as plt
import os
import sys
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
import sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
torch.manual_seed(13)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_height = 9
input_width = 9
input_channel_num = 4
# conv_fuse = "plus"

# conv_1_shape = '4*4*1*16'
# pool_1_shape = 'None'

# # conv_2_shape = 'None'
# conv_2_shape = '4*4*1*32'
# pool_2_shape = 'None'

# # conv_3_shape = 'None'
# conv_3_shape = '4*4*1*64'
# pool_3_shape = 'None'

# conv_4_shape = '1*1*128*4'
# pool_4_shape = 'None'

time_step = 1
window_size = 1
# convolution full connected parameter
fc_size = 1024

dropout_prob = 0.5
np.random.seed(3)

calibration = 'N'
norm_type = '2D'
regularization_method = 'dropout'
enable_penalty = True

In [ ]:
arousal_key = "arousal_labels"
valence_key = "valence_labels"

# def minus(item):
#     return item-1

def construct_data(input_file, arousal_or_valence=None, bands=[0, 1, 2, 3]):
    input_channel_num = len(bands) * time_step

    dataset_dir = '/content/drive/MyDrive/DEAP_3D//DE3D_'
    ###load training set
    print("loading ",dataset_dir+input_file,".mat")
    data_file = sio.loadmat(dataset_dir+input_file+".mat")


    cnn_datasets = data_file["data"]

    #start
    arousal_labels = data_file[arousal_key]
    valence_labels = data_file[valence_key]

    arousal_labels = np.where(arousal_labels == 1, 2, arousal_labels)

    curr_labels = np.add(arousal_labels, valence_labels)

    # 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
    labels = curr_labels[0]

    # print('labels: ', np.unique(labels, return_counts= True))

    cnn_datasets = cnn_datasets.transpose(0,2,3,1)

    cnn_datasets = cnn_datasets[:,:,:,bands]

    cnn_datasets = cnn_datasets.reshape(len(cnn_datasets)//time_step, window_size,input_height,input_width,input_channel_num)

    index = np.array(range(0, len(labels)))
    np.random.shuffle(index)

    cnn_datasets   = cnn_datasets[index].reshape((-1, 4, 9, 9))
    labels  = labels[index]

    return cnn_datasets, labels  

In [ ]:
class DatasetGenerator(Dataset):
    def __init__(self, data_X, data_y):
        self.data = data_X
        self.labels = data_y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        inst = self.data[idx]
        label = self.labels[idx]
        return inst, label

In [2]:
class NnModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(4, 32, (3, 3), 1, 1)
        # self.conv2 = nn.Conv2d(32, 64, (3, 3), 1, 1)
        # self.conv3 = nn.Conv2d(64, 32, (3, 3), 1, 1)
        # self.conv4 = nn.Conv2d(128, 64, (1, 1))
        self.fc1 = nn.Linear(4 * 9 * 9, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 4)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.relu(self.conv3(x))
        # x = F.relu(self.conv4(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = NnModel()
model

NnModel(
  (fc1): Linear(in_features=324, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=4, bias=True)
)

In [ ]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)

    confusion_matrix = torch.zeros(4, 4)
    with torch.no_grad():  
      for t, p in zip(y_test.view(-1), y_pred_tags.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
    
    acc = round(acc.item() * 100, 2)
    
    return acc, confusion_matrix

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def report_result(best_model, data_X, data_y, label):
  print('\n',label,'\n----------------\n')
  nb_classes = 4
  data_y_hat = best_model.forward(data_X)
  data_acc, confusion_matrix = multi_acc(data_y_hat, data_y)
  print('****** mean classification accuracy: ', round(data_acc, 2), '% ******')
  # print(confusion_matrix)
  print('\n Accuracy per class\n ---------------')
  acc_per_class = np.array(confusion_matrix.diag()/confusion_matrix.sum(1))
  for i in range(len(acc_per_class)):
    print(label_mapping[i], ': ', round(acc_per_class[i]*100, 2), '%')
  print('****** mean per class accuracy: ', round(np.mean(acc_per_class)*100, 2),'% ******\n')

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def get_result(best_model, data_X, data_y, label):
  ret = {}
  nb_classes = 4
  data_y_hat = best_model.forward(data_X)
  data_acc, confusion_matrix = multi_acc(data_y_hat, data_y)
  ret['acc'] = round(data_acc, 2)
  acc_per_class = np.array(confusion_matrix.diag()/confusion_matrix.sum(1))
  for i in range(len(acc_per_class)):
    ret[label_mapping[i]] = round(acc_per_class[i]*100, 2)
  ret['per_class_acc'] = round(np.mean(acc_per_class)*100, 2)

  return ret

In [ ]:
def train_model(model, train_X, train_y, valid_X, valid_y, batch=64, epoch_=50):
    train_dataset = DatasetGenerator(train_X, train_y)
    dataloader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

    epochs = epoch_
    loss_arr = []
    acc_arr = []
    val_loss_arr = []
    val_acc_arr = []
    best_val_acc = -1

    for i in range(epochs):
        for j, (batch_X, batch_y) in enumerate(dataloader): 
          y_hat = model.forward(batch_X)
          loss = criterion(y_hat, batch_y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()  
        
        y_hat = model.forward(train_X)
        train_loss = criterion(y_hat, train_y)
        train_acc = multi_acc(y_hat, train_y)[0]
        valid_y_hat = model.forward(valid_X)
        valid_loss = criterion(valid_y_hat, valid_y)
        valid_acc = multi_acc(valid_y_hat, valid_y)[0]
        
        loss_arr.append(train_loss.detach())
        acc_arr.append(train_acc)
        val_loss_arr.append(valid_loss.detach())
        val_acc_arr.append(valid_acc)
        
        if valid_acc > best_val_acc:
          best_val_acc = valid_acc
          torch.save(model.state_dict(), 'best-model-parameters.pt')
          print(f'[Best Model Saved] Epoch: {i+1} Train Loss: {train_loss} Train Acc: {train_acc} Valid Loss: {valid_loss} Valid Acc: {valid_acc}')
        else: #elif (i+1) % 10 == 0:
          print(f'Epoch: {i+1} Train Loss: {loss} Train Acc: {train_acc} Valid Loss: {valid_loss} Valid Acc: {valid_acc}')
        
    return {'train_loss': loss_arr, 'val_loss': val_loss_arr}

In [ ]:
{0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
acc_list = []
pc_acc_list = []
lalv_acc_list = []
lahv_acc_list = []
halv_acc_list = []
hahv_acc_list = []

for i in range(16):
  sub_id = i + 1
  print("**************** Process starting for subject no. ", sub_id, "**************** \n")
  if sub_id < 10:
    input_file = 's0' + str(sub_id)
  else:
    input_file = 's' + str(sub_id)
  model = NnModel()
  data, labels = construct_data(input_file)
  X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
  train_X = torch.FloatTensor(X_train)
  train_y = torch.LongTensor(y_train)
  valid_X = torch.FloatTensor(X_val)
  valid_y = torch.LongTensor(y_val)
  test_X = torch.FloatTensor(X_test)
  test_y = torch.LongTensor(y_test)
  _ = train_model(model, train_X, train_y, valid_X, valid_y, epoch_=30)
  best_model = CnnModel()
  best_model.load_state_dict(torch.load('/content/best-model-parameters.pt'))
  ret = get_result(best_model, test_X, test_y, 'Test Set')
  acc_list.append(ret['acc'])
  pc_acc_list.append(ret['per_class_acc'])
  lalv_acc_list.append(ret['LALV'])
  lahv_acc_list.append(ret['LAHV'])
  halv_acc_list.append(ret['HALV'])
  hahv_acc_list.append(ret['HAHV'])

**************** Process starting for subject no.  1 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s01 .mat
[Best Model Saved] Epoch: 1 Train Loss: 1.2099146842956543 Train Acc: 47.99 Valid Loss: 1.219802975654602 Valid Acc: 43.96
[Best Model Saved] Epoch: 2 Train Loss: 1.0399264097213745 Train Acc: 60.28 Valid Loss: 1.0741469860076904 Valid Acc: 55.62
[Best Model Saved] Epoch: 3 Train Loss: 0.8539854884147644 Train Acc: 72.08 Valid Loss: 0.9167521595954895 Valid Acc: 62.29
[Best Model Saved] Epoch: 4 Train Loss: 0.6771066188812256 Train Acc: 80.49 Valid Loss: 0.7763875126838684 Valid Acc: 72.92
[Best Model Saved] Epoch: 5 Train Loss: 0.5407343506813049 Train Acc: 85.42 Valid Loss: 0.6654956340789795 Valid Acc: 75.42
[Best Model Saved] Epoch: 6 Train Loss: 0.4182530343532562 Train Acc: 88.61 Valid Loss: 0.5446787476539612 Valid Acc: 80.83
[Best Model Saved] Epoch: 7 Train Loss: 0.3386352062225342 Train Acc: 91.6 Valid Loss: 0.4608481228351593 Valid Acc: 85.21
[Best M

In [ ]:
result_dict =   {'acc': acc_list,
  'pc_acc': pc_acc_list,
  'lalv': lalv_acc_list,
  'lahv': lahv_acc_list,
  'halv': halv_acc_list,
  'hahv': hahv_acc_list}

In [ ]:
result_dict

{'acc': [93.33,
  79.58,
  87.92,
  87.71,
  76.25,
  90.83,
  93.54,
  88.75,
  90.21,
  90.21,
  80.83,
  88.75,
  95.21,
  87.29,
  92.92,
  92.08],
 'hahv': [94.41,
  81.48,
  90.0,
  88.03,
  78.11,
  92.76,
  93.4,
  92.02,
  89.41,
  90.76,
  77.78,
  92.44,
  96.89,
  82.31,
  96.97,
  97.96],
 'halv': [92.44,
  83.7,
  82.72,
  76.0,
  65.15,
  86.54,
  94.19,
  76.47,
  89.38,
  87.59,
  79.31,
  85.03,
  95.22,
  91.05,
  91.24,
  90.23],
 'lahv': [90.24,
  63.38,
  93.65,
  76.0,
  80.34,
  92.86,
  94.31,
  91.67,
  89.36,
  90.27,
  88.77,
  90.0,
  95.24,
  86.87,
  93.13,
  88.0],
 'lalv': [94.92,
  82.81,
  80.29,
  92.44,
  75.78,
  83.33,
  91.53,
  92.44,
  92.0,
  92.79,
  72.79,
  85.29,
  86.11,
  86.89,
  91.15,
  91.95],
 'pc_acc': [93.0,
  77.84,
  86.66,
  83.12,
  74.85,
  88.87,
  93.35,
  88.15,
  90.04,
  90.35,
  79.66,
  88.19,
  93.36,
  86.78,
  93.12,
  92.03]}

In [ ]:
np.mean(np.array(result_dict['acc']))

88.46312499999999

In [ ]:
np.save('result_nn.npy', result_dict)